In [4]:
import pandas as pd
import folium
import networkx as nx
from math import sqrt

# Load processed file
df = pd.read_csv("./data/processed_bangalore_crime.csv")

# Keep required columns
df = df[['Latitude', 'Longitude', 'risk_score']]

# 🔥 IMPORTANT FIX — Convert to numeric
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['risk_score'] = pd.to_numeric(df['risk_score'], errors='coerce')

df = df.dropna()

# Optional: restrict to Bangalore bounding box
df = df[
    (df['Latitude'].between(12.7, 13.2)) &
    (df['Longitude'].between(77.4, 77.8))
]

grid_size = 0.01

df['lat_grid'] = (df['Latitude'] // grid_size) * grid_size
df['lon_grid'] = (df['Longitude'] // grid_size) * grid_size

grid_risk = df.groupby(['lat_grid', 'lon_grid'])['risk_score'].mean().reset_index()

# Assign cost for Dijkstra
def zone_and_cost(score):
    if score <= 2:
        return 'Safe', 1
    elif score <= 4:
        return 'Moderate', 3
    else:
        return 'High Risk', 10

grid_risk[['zone', 'cost']] = grid_risk['risk_score'].apply(
    lambda x: pd.Series(zone_and_cost(x))
)

# ----------------------------
# Build Graph
# ----------------------------
G = nx.Graph()

for _, row in grid_risk.iterrows():
    G.add_node(
        (row['lat_grid'], row['lon_grid']),
        cost=row['cost'],
        zone=row['zone']
    )

def distance(a, b):
    return sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

nodes = list(G.nodes)
K = 5

for node in nodes:
    neighbors = sorted(nodes, key=lambda x: distance(node, x))[1:K+1]
    for nbr in neighbors:
        weight = G.nodes[nbr]['cost']
        G.add_edge(node, nbr, weight=weight)

# ----------------------------
# Example Bangalore locations
# ----------------------------
start_raw = (12.9716, 77.5946)  # MG Road
end_raw   = (12.9352, 77.6245)  # Koramangala

def nearest_node(point):
    return min(nodes, key=lambda n: (n[0] - point[0])**2 + (n[1] - point[1])**2)

start = nearest_node(start_raw)
end = nearest_node(end_raw)

safe_path = nx.dijkstra_path(G, start, end, weight='weight')

# ----------------------------
# Visualize
# ----------------------------
m = folium.Map(location=[12.9716, 77.5946], zoom_start=13)

folium.PolyLine(
    locations=safe_path,
    color='green',
    weight=5
).add_to(m)

folium.Marker(start, icon=folium.Icon(color='blue')).add_to(m)
folium.Marker(end, icon=folium.Icon(color='red')).add_to(m)

m.save("./output/bangalore_safe_path.html")

print("✅ Bangalore safe path generated successfully.")

✅ Bangalore safe path generated successfully.
